In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import sieve
import time

from common import *
#from solver

In [2]:
p = problem(base_path='./solution/base3.csv')
p.score(p.base,raw=False)

1516562.8799840237

In [23]:
p.base = solution
p.solve()

Start iteration 1
a: 204 | saving: -2.115 | cumsave: -2.115
a: 744 | saving: -1.165 | cumsave: -3.280
a: 746 | saving: -4.001 | cumsave: -7.282
a: 3691 | saving: -2.414 | cumsave: -9.696
a: 3691 | saving: -15.142 | cumsave: -24.838
a: 17745 | saving: -1.998 | cumsave: -26.836
a: 17745 | saving: -1.330 | cumsave: -28.166
a: 22864 | saving: -1.473 | cumsave: -29.639
a: 28902 | saving: -0.073 | cumsave: -29.712
a: 30774 | saving: -1.732 | cumsave: -31.444
a: 30779 | saving: -0.182 | cumsave: -31.626
a: 31006 | saving: -0.666 | cumsave: -32.291
a: 37338 | saving: -0.130 | cumsave: -32.422
a: 39860 | saving: -5.553 | cumsave: -37.975
a: 39860 | saving: -5.238 | cumsave: -43.214
a: 39876 | saving: -0.326 | cumsave: -43.540
a: 39876 | saving: -2.336 | cumsave: -45.876
a: 44011 | saving: -0.279 | cumsave: -46.155
a: 48331 | saving: -4.292 | cumsave: -50.447
a: 52487 | saving: -0.189 | cumsave: -50.636
a: 52805 | saving: -0.683 | cumsave: -51.320
a: 52807 | saving: -0.267 | cumsave: -51.587
a: 

In [3]:
solution = p.base
nodes = p.nodes
prime = p.prime
prime_set = p.prime_set
n = len(nodes)
not_prime = np.logical_not(np.isin(solution,prime))

In [4]:
a = time.time()
best = []
for i,node in enumerate(nodes):
    best_d = distance(node,nodes)
    best_d[i] = 9999
    best.append(best_d.min())
print(time.time()-a)

647.9612607955933


In [627]:
a = time.time()
good = 0
goods = []
dist = score(nodes,solution,prime,raw=True)
for i in range(len(nodes)):
    left_node = solution[i]
    right_node = solution[i+1]
    
    link = dist[i]
    left_best = best[left_node]
    right_best = best[right_node]
    
    left_prime = left_node not in p.prime_set
    is_ten = (i+1)%10 == 0
    if left_prime and is_ten:
        left_best = left_best*1.0
    
    is_good = link<=left_best or link<=right_best
    if is_good:
        good+=1
    goods.append(is_good)
print(good)
print(time.time()-a)

123087
0.30493879318237305


In [628]:
bads = np.logical_not(np.array(goods))

In [629]:
targets = np.argwhere(bads).ravel()

a=time.time()
candidates = []
candidates_ = []
for left in targets[0:10000]:
    current_d = dist[left]
    
    right = left+1
    left_node,right_node = solution[[left,right]]
    
    left_d = distance(nodes[left_node],nodes)
    right_d = distance(nodes[right_node],nodes)
    
    left_n = (left_d<current_d).sum()-1
    right_n = (right_d<current_d).sum()-1
    
    left_d[left_node] = 9999
    right_d[right_node] = 9999
    
    left_room = np.argwhere(left_d<current_d).ravel()
    right_room = np.argwhere(right_d<current_d).ravel()
    candidates.append((left,left_room,right_room))
    
print(time.time()-a)

real_candidates = []
for current,left,right in candidates:  
    left_pos = []
    right_pos = []
    for node in left:
        
        left_pos.append(np.argwhere(solution==node)[0,0]-1)
    for node in right:
        right_pos.append(np.argwhere(solution==node)[0,0]-1)
    real_candidates.append((current,left_pos,right_pos))

137.18729543685913


In [56]:
def which_one(four_points):
    b,c,d,e = four_points
    first_reverse_pos = 197769-c
    second_reverse_pos = 197769-e

    first_switch_pos = b+e-d+1
    
    first_non_for_off = 0
    first_swi_for_off = (first_switch_pos-b)%10
    first_non_back_off = (b - first_reverse_pos)%10+10
    first_swi_back_off = (first_switch_pos - first_reverse_pos)%10+10
    
    second_non_for_off = 0
    second_swi_for_off = (b-d)%10
    second_non_back_off = (d - second_reverse_pos)%10+10
    second_swi_back_off = (b - second_reverse_pos)%10+10
    
    return [[first_non_for_off,first_swi_for_off,first_non_back_off,first_swi_back_off], 
            [second_non_for_off,second_swi_for_off,second_non_back_off,second_swi_back_off]]

def connector_dist(row,nodes):
    return distance(nodes[row[::2]],nodes[row[1::2]])

def segment_dist(which_ones,four_points,dist):
    which_1,which_2 = which_ones
    b,c,d,e = four_points
    
    segment_1 = dist[which_1,b:c].sum(axis=-1)
    segment_2 = dist[which_2,d:e].sum(axis=-1)
    segment = segment_1[[0,0,2,2,3,1,1,3]] + segment_2[[0,2,0,2,1,1,3,3]] 
    
    return segment


def check(links,nodes,solution,prime_set,dist):
    a,b = links[0],links[0]+1
    c,d = links[1],links[1]+1
    e,f = links[2],links[2]+1
    
    four_points = (b,c,d,e)
    
    a_,b_,c_,d_,e_,f_ = solution[[a,b,c,d,e,f]]
    a_p = not a_ in prime_set
    b_p = not b_ in prime_set
    c_p = not c_ in prime_set
    d_p = not e_ in prime_set
    e_p = not e_ in prime_set
    
    a_ten = b%10==0
    c_ten = d%10==0
    e_ten = f%10==0
    m_ten = (b+e-d+1)%10==0
    
    check = [[a_,b_,c_,d_,e_,f_],
             [a_,b_,c_,e_,d_,f_],
             [a_,c_,b_,d_,e_,f_],
             [a_,c_,b_,e_,d_,f_],
             [a_,d_,e_,c_,b_,f_],
             [a_,d_,e_,b_,c_,f_],
             [a_,e_,d_,b_,c_,f_],
             [a_,e_,d_,c_,b_,f_]]   
    first = a_ten and a_p
    mask = [[first, c_ten and c_p, e_ten and e_p],
            [first, c_ten and c_p, e_ten and d_p],
            [first, c_ten and b_p, e_ten and e_p],
            [first, c_ten and b_p, e_ten and d_p],
            [first, m_ten and e_p, e_ten and b_p],
            [first, m_ten and e_p, e_ten and c_p],
            [first, m_ten and d_p, e_ten and c_p],
            [first, m_ten and d_p, e_ten and b_p]]
    mask = np.array(mask)*0.1
    connector = []
    for row in check:
        connector.append(connector_dist(row,nodes))
    connector = np.array(connector)
    connector = (connector+connector*mask).sum(axis=-1)
    
    which_ones = which_one(four_points)
    segment = segment_dist(which_ones,four_points,dist)
    
    out = connector+segment
    return np.argmin(out),out.min()-out[0]

def excute_swap(code,three_points,solution):
    b,d,f = three_points
    first = solution[b:d].copy()
    second = solution[d:f].copy()
    
    if code == 1:
        solution[b:f] = np.concatenate([first,second[::-1]])
        return
    elif code == 2:
        solution[b:f] = np.concatenate([first[::-1],second])
        return
    elif code == 3:
        solution[b:f] = np.concatenate([first[::-1],second[::-1]])
        return
    elif code == 4:
        solution[b:f] = np.concatenate([second,first[::-1]])
        return
    elif code == 5:
        solution[b:f] = np.concatenate([second,first])
        return
    elif code == 6:
        solution[b:f] = np.concatenate([second[::-1],first])
        return
    elif code == 7:
        solution[b:f] = np.concatenate([second[::-1],first[::-1]])
        return
    
def two_way_distance(nodes,solution,prime):
    s = nodes[solution[:-1]]
    e = nodes[solution[1:]]
    raw_d = distance(s,e)
    rever_raw_d = raw_d[::-1]
    
    n = len(raw_d)
    dist = np.zeros((20,n),dtype=float)
    
    notprime = np.logical_not(np.isin(solution,prime))
    notprime_forward = notprime[:-1]
    notprime_backward = notprime[-1:0:-1]
    
    for offset in range(10):
        dist[offset,9-offset::10] += raw_d[9-offset::10] * notprime_forward [9-offset::10] * 0.1
        dist[offset+10,9-offset::10] += (rever_raw_d[9-offset::10] * notprime_backward[9-offset::10] * 0.1)
        dist[offset+10] = dist[offset+10][::-1]
    
    return dist

In [57]:
p = problem(base_path='./solution/base3.csv')
solution = p.base.copy()
nodes = p.nodes.copy()
prime = p.prime.copy()
prime_set = p.prime_set
n = len(nodes)
not_prime = np.logical_not(np.isin(solution,prime))

links = (77,78,110)
dist = two_way_distance(nodes,solution,prime)                    
check(links,nodes,solution,prime_set,dist)

(4, -0.92529802278737)

In [48]:
which_one((78,78,79,110))

[[0, 2, 17, 19], [0, 9, 10, 19]]

In [58]:
a=time.time()

dist = two_way_distance(nodes,solution,prime)
swaps = 1
iteration = 1
while swaps>0:
    swaps = 0
    iterstart = time.time()
    print('iteration %s start'%iteration)
    right_d = distance(nodes[0],nodes)
    right_d[0] = 99999

    for left in range(n):
        left_node,right_node = solution[left:left+2]
        left_prime = not left_node in prime_set

        current_d = distance(nodes[left_node], nodes[right_node])

        left_d = right_d.copy()
        right_d = distance(nodes[right_node],nodes)
        right_d[right_node] = 99999

        left_candidates = np.argwhere(left_d<current_d).ravel()
        right_candidates = np.argwhere(right_d<current_d).ravel()

        if len(left_candidates)==0 or len(right_candidates)==0:
            continue
        for node_l in left_candidates:

            ll = np.argwhere(solution==node_l)[0,0]
            for node_r in right_candidates:
                rr = np.argwhere(solution==node_r)[0,0]

                for l,r in [(ll,rr),(ll-1,rr),(ll,rr-1),(ll-1,rr-1)]:
                    a,c,e = np.sort([left,l,r])

                    if c-a>0 and e-c>0:
                        #num+=1
                        code,improvement = check((a,c,e),nodes,solution,prime_set,dist)
                        if code>0:
                            swaps += 1
                            excute_swap(code,(a+1,c+1,e+1),solution)
                            dist = two_way_distance(nodes,solution,prime)
                            print(a,c,e,code,improvement,p.score(solution))
    iterend = time.time()
    iterduation = iterend-iterstart
    print('iteration %s end, swaps %s, time lapse %s'%(iteration,swaps,iterduation))
    print('###############################')
    iteration += 1
print('done')

iteration 1 start
77 78 110 4 -0.92529802278737 1516561.9546860007
230 233 197751 7 -7.180140354774267 1516554.7745456456
280 548 549 5 -2.211911483581659 1516552.5626341621
370 407 408 5 -0.00808006593028665 1516552.5545540962
676 679 685 4 -0.3142380996443066 1516552.2403159966


KeyboardInterrupt: 

In [672]:
np.unique(solution).shape

(197769,)

In [664]:
190000**2/num/200

61.21322924710441

In [639]:
num = 0
dist = two_way_distance(nodes,solution,prime)
for current,left,right in real_candidates[:10000]:
    for l in left:
        for r in right:
            a,c,e = np.sort([current,l,r])
            
            if c-a>=2 and e-c>=2:
                num+=1
                code,improvement = check((a,c,e),nodes,solution,prime_set,dist)
                if code>0:
                    excute_swap(code,(a+1,c+1,e+1),solution)
                    dist = two_way_distance(nodes,solution,prime)
                    print(a,c,e,code,improvement,p.score(solution))

In [427]:
p.score(solution)-1516562.8799840237

0.0

# UTILITY

In [41]:
#write result
pd.DataFrame({'Path':solution}).to_csv('./solution/base3.csv',index=False)

In [39]:
#read LKH result into solution
f = open('./solver/tour_alpha_10.txt','r')
line = 1
content = '123'
sol = []
while content != '':
    content = f.readline()
    if line >=7:
        
        sol.append(content)
    line+=1
f.close()

sol = sol[:-2]
sol = [int(line.split('\n')[0]) for line in sol]
sol[-1] = 1
solution = np.array(sol)
solution-=1

In [ ]:
n=p.n
nodes = p.nodes

In [ ]:
#make .tsp problem
fname =  './problem/santa_.tsp'
with open(fname, 'w') as f:
    f.write('NAME : santa\n')
    f.write('COMMENT : santa\n')
    f.write('TYPE : TSP\n')
    f.write('DIMENSION : %d\n' % n)
    f.write('EDGE_WEIGHT_TYPE : EUC_2D\n')
    f.write('NODE_COORD_SECTION\n')
    for idx,node in enumerate(nodes):
        x,y=node*1000
        x = int(x)
        y=int(y)
        f.write('%s %s %s\n' % (idx+1,x,y))
    f.write('EOF\n')

In [ ]:
#read neos html
fname = path + 'solution/neos2.html'
l = 1
sol = []
with open(fname, 'r') as f:
    while l< 207274:
        line = f.readline()
        if l>=9505:
            sol.append(int(line.split(' ')[0]))
        l += 1
sol.append(0)
solution = np.array(sol)

In [ ]:
#2-opt manual check calcucation
a,d = 185337,186129 
a_pre,d_nex = a-1,d+1

node_a,node_d,node_a_pre,node_d_nex = solution[[a,d,a_pre,d_nex]]
old = score(solution)[a_pre:d_nex].sum()

to_d  = distance(nodes[node_a_pre],nodes[node_d])
from_a = distance(nodes[node_a],nodes[node_d_nex])
check = (d+1)%10 ==0 and node_a not in prime
print(check)
if check:
    from_a*=1.1

partial = solution[185337:186129+1][::-1]
total = score(partial,offset = a%10).sum()

In [ ]:
#2-opt functions
def calc_reverse(left, nodes, solution):
    r_solution = solution[left:-1][::-1]
    reverse_dist = []
    for offset in range(10):
        reverse_dist.append(np.cumsum(score(nodes,r_solution,offset)[::-1])[::-1])
    reverse_dist = np.array(reverse_dist)
    
    return reverse_dist

def calc_old_dist(nodes, solution, from_node):
    offset = from_node%10
    return np.cumsum(score(nodes,solution[from_node:],offset=offset))

def calc_middle(reverse_dist, left):
    middle = np.zeros(shape=(reverse_dist.shape[1]))
    offset = left%10
    
    for o in range(10):
        order = (offset-o)%10
        using_reverse = reverse_dist[order]
        middle[o::10] = using_reverse[o::10]
    
    reverse_dist[:,:-1] -= reverse_dist[:,-1:]
    reverse_dist = reverse_dist[:,:-1]
    return middle[::-1], reverse_dist

def calc_connector(nodes, solution_nodes, roller, left, node_before_a, node_a, prime_set):
    to_d = roller[:-1].copy()
    offset = left%10
    if node_before_a not in prime_set and offset == 0:
        to_d *= 1.1
        
    roller = distance(nodes[node_a],solution_nodes[left+2:])
    if node_a in prime_set:
        return to_d, roller, roller  
    else:
        start_leg = (10-left+2)%10
        from_a = roller.copy()
        from_a[start_leg::10] *= 1.1
        return to_d, from_a, roller
    
def calc_save(from_a,to_d,middle,old_dist):
    save = from_a+to_d-old_dist[2:]+middle
    old_dist[1:] -= old_dist[0]
    old_dist = old_dist[1:]
    
    place = np.argmin(save)
    saving = save[place]
    return place, saving, old_dist

def calc_basis(start_at, nodes, solution, solution_nodes):
    reverse_dist = calc_reverse(start_at, nodes, solution)
    old_dist = calc_old_dist(nodes, solution,from_node=start_at-1)
    roller = distance(nodes[solution[start_at-1]],solution_nodes[start_at-1+2:])
    
    return reverse_dist, old_dist, roller

def excute_swap(left,place,solution,solution_nodes):
    target = place+1+left
    
    solution[left:target+1] = solution[target:left-1:-1]
    solution_nodes[left:target+1] = solution_nodes[target:left-1:-1]
    

In [ ]:
bests = []
dids = []

asd = nodes[solution]
df = pd.DataFrame(asd)
df.to_csv('asd.csv')
solution_nodes = pd.read_csv('asd.csv').iloc[:,1:].values

start = time.time()

i=1
iter_num = 5
start_at = 1
run_through = 2000

fast = 

while i <= iter_num:
    iter_start = time.time()
    best = []
    did = 0
    
    reverse_dist, old_dist, roller = calc_basis(start_at, nodes, solution, solution_nodes)

    for left in range(start_at, run_through):
        node_before_a, node_a = solution[left-1:left+1]

        middle, reverse_dist = calc_middle(reverse_dist, left)

        to_d, from_a, roller =  calc_connector(nodes, solution_nodes, roller, left, node_before_a, node_a, prime_set)

        place, saving, old_dist = calc_save(from_a,to_d,middle,old_dist)
        
        if saving<0:
            
            best.append(saving)
            did += saving
            target = place+1+left
            
            #execute swap
            excute_swap(left,place,solution,solution_nodes)
            print('a: %s | saving: %.3f | cumsave: %.3f'%(left, saving, did))
            
            #reset infos
            reverse_dist, old_dist, roller = calc_basis(left+1, nodes, solution, solution_nodes)

    #pd.DataFrame({'Path':solution}).to_csv('./solution/neos_iter_%s.csv'%(i+1),index=False)
    bests.append(best)
    dids.append(did)
    print('iter %s done, total swap %s, saves %.3f, time %.1f'%(i,len(best),did,time.time()-iter_start))
    print(score(nodes,solution).sum())
    print('####################################')
    i+=1
    if len(best) == 0:
        print('zero')
        break
print(time.time()-start)

# node swap

In [24]:

nodes = p.nodes
solution = p.solution
prime = p.prime
n = len(nodes)

In [665]:
start = time.time()

#every iter
coor = nodes[solution]
dist = score(nodes,solution,prime)
neighbor = pd.DataFrame({'d':dist}).d.rolling(2,center=True).sum().values
oldnext = pd.DataFrame({'d':dist}).d.rolling(3,center=True).sum().values
notprime = ~np.isin(solution[:-1], prime)
isten = (np.arange(n)+1)%10==0
isextra = isten&notprime
better = 0

time_a = time.time()

cur = np.append([0],distance(coor[0],coor))
nex = np.append([0],distance(coor[1],coor))

swap = {}
for left in range(1,n-1):
#every left
    pre_coor, left_coor, next_coor = coor[left-1:left+2]
    adj_dist = distance(left_coor, next_coor)
    
    FROM_FLAG = (left+1)%10 == 0
    TO_FLAG = left%10 == 0
    LEFT_FLAG = notprime[left]
    LEFT_pre_FLAG = notprime[left-1]
    LEFT_nex_FLAG = notprime[left+1]
    
    ##old cost
    left_nb = neighbor[left]
    old_cost = neighbor[left:] + left_nb
    old_cost[0] = 0
    old_cost[1] = oldnext[left]

    ##new cost
    pre = cur[1:]
    
    cur = nex[1:]
    
    nex = distance(next_coor,coor[left-1:])

    cur_1 = cur[:-2]
    cur_2 = cur[2:]
    pre_ = pre[1:-1]
    nex_ = nex[1:-1]
    
    if TO_FLAG & LEFT_pre_FLAG:
        pre_ = pre_ + pre_*0.1
    if FROM_FLAG:
        nex_ = nex_ + nex_*notprime[left:]*0.1
        
    cur_1 = cur_1 + cur_1*isextra[left-1:-1]*0.1
    
    if LEFT_FLAG:
        cur_2 = cur_2 + cur_2*isten[left:]*0.1
    
    if FROM_FLAG & LEFT_nex_FLAG:
        adj_dist *= 1.1
    new_cost = pre_ + nex_ + cur_1 + cur_2
    
    
    new_cost[0] = 0
    new_cost[1] += adj_dist
    
    save = old_cost - new_cost
    good = np.where(save>0)[0]
    
    if len(good)>0:
        good_save = save[good]
        swap[left] = [good+left,good_save]
    
    if left%10000 == 0:
        time_b = time.time()
        print(left,time_b-time_a)
        time_a = time_b
print(time.time()-start)
print(len(swap))
###############################################
bk = set()
candidate = []
for left in swap:
    right,improve = swap[left]
    for i in range(len(right)):
        candidate.append((improve[i],right[i],left))
        
candidate = sorted(candidate)[::-1]
l=[]
r=[]
for improve,left,right in candidate:
    fill = [left-1,left,left+1,right-1,right,right+1]
    fill = [not (x in bk) for x in fill]
    if all(fill):
        l.append(left)
        r.append(right)
        bk.add(left-1)
        bk.add(left)
        bk.add(left+1)
        bk.add(right-1)
        bk.add(right)
        bk.add(right+1)

buf = solution[l]
solution[l] = solution[r]
solution[r] = buf

dist = score(nodes,solution,prime).sum()

10000 123.50499987602234
20000 115.93300008773804
30000 108.6489999294281
40000 103.66299986839294
50000 96.67000031471252
60000 89.70099973678589
70000 77.0220000743866
80000 54.45700001716614
90000 48.789000034332275
100000 43.92300009727478
110000 39.602999687194824
120000 35.05200004577637
130000 29.98799991607666
140000 23.11400032043457
150000 18.20799994468689
160000 14.726999998092651
170000 10.738999843597412
180000 7.234000205993652
190000 4.010999917984009
1046.2929999828339
11


In [666]:
candidate

[(5.512188079260476, 189095, 96583),
 (5.313985566707899, 39720, 769),
 (4.014918448782723, 82685, 61057),
 (3.465779936194707, 45401, 39871),
 (1.1158088994012303, 133517, 109051),
 (0.86968993612453, 60012, 48341),
 (0.7396648432312389, 145139, 145138),
 (0.46492546636212495, 197750, 230),
 (0.33728463005964215, 162566, 161279),
 (0.3166854086281887, 91110, 91109),
 (0.03957997718850237, 57179, 57178)]

In [667]:
p.score(solution)

1516277.1968996653